
## LangGraph教程


-----

### **背景介绍**

在 LangChain，我们的目标是让构建大型语言模型（LLM）应用变得简单。您可以构建的一种 LLM 应用就是**智能代理（Agent）**。构建智能代理非常令人兴奋，因为它们能够自动化以前不可能完成的各种任务。

然而，在实践中，构建能够可靠执行这些任务的系统极其困难。在我们与用户合作将智能代理投入生产的过程中，我们发现通常需要更多的**控制**。例如，您可能需要智能代理始终优先调用某个特定的工具，或者根据其状态使用不同的提示词。

为了解决这个问题，我们构建了 [**LangGraph**](https://langchain-ai.github.io/langgraph/) —— 一个用于构建智能代理和多智能体应用的框架。LangGraph 独立于 LangChain 包，其核心设计理念是帮助开发者为智能代理工作流添加更好的**精确性**和**控制力**，使其适合现实世界系统的复杂性。

-----

### **课程结构**

本课程由一系列模块组成，每个模块都专注于一个与 LangGraph 相关的主题。您会看到每个模块都有一个文件夹，其中包含一系列**笔记本（notebooks）**。每本笔记本都配有视频，以帮助您理解概念，但这些笔记本也是独立的，这意味着它们包含了详细的解释，可以脱离视频独立观看。每个模块文件夹还包含一个 `studio` 文件夹，其中包含一组图（graphs），这些图可以加载到 [**LangGraph Studio**](https://github.com/langchain-ai/langgraph-studio) 中，这是我们用于构建 LangGraph 应用的集成开发环境（IDE）。

-----

### **设置**

在开始之前，请按照 `README` 文件中的说明创建环境并安装依赖项。

-----

### **聊天模型**

在本课程中，我们将使用[**聊天模型（Chat Models）**](https://python.langchain.com/v0.2/docs/concepts/#chat-models)，它们的功能是接收一系列消息作为输入，并以聊天消息作为输出。LangChain 本身不托管任何聊天模型，而是依赖于第三方集成。 是 LangChain 中支持的第三方聊天模型集成列表！默认情况下，课程将使用[ChatOpenAI](https://python.langchain.com/v0.2/docs/integrations/chat/openai/)  ，因为它既流行又性能出色。正如前面提到的，请确保您已设置好 `OPENAI_API_KEY`。

我们将检查您的 `OPENAI_API_KEY` 是否已设置；如果未设置，系统会提示您输入。

In [1]:
# 安装必要的依赖包
# %%capture --no-stderr 用于隐藏安装过程中的输出信息
# %pip install 是 Jupyter 中安装 Python 包的命令
# --quiet 参数减少输出信息
# -U 参数表示升级到最新版本
%%capture --no-stderr
%pip install --quiet langchain_openai==0.3.32 langchain_core==0.3.75 langchain_community==0.3.29 tavily-python==0.7.11

In [2]:
%pip show  langchain_openai langchain_core langchain_community tavily-python

Name: langchain-openai
Version: 0.3.32
Summary: An integration package connecting OpenAI and LangChain
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: langchain-core, openai, tiktoken
Required-by: 
---
Name: langchain-core
Version: 0.3.75
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: jsonpatch, langsmith, packaging, pydantic, PyYAML, tenacity, typing-extensions
Required-by: langchain, langchain-community, langchain-openai, langchain-text-splitters
---
Name: langchain-community
Version: 0.3.29
Summary: Community contributed LangChain integrations.
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: aiohttp, dataclasses-json, httpx-sse, langchain, langchain-core, langsmith, numpy, pydantic-settings, PyYAML, requests, SQLAlchemy, tenacity
Required-b

In [6]:
# 导入必要的模块
import os, getpass

def _set_env(var: str):
    """
    设置环境变量的辅助函数

    参数:
        var (str): 要设置的环境变量名称

    功能:
        - 检查环境变量是否已存在
        - 如果不存在，则提示用户输入并设置
    """
    if not os.environ.get(var):  # 检查环境变量是否已设置
        os.environ[var] = getpass.getpass(f"{var}: ")  # 安全地获取用户输入

# 设置 OpenAI API 密钥
# 这是使用 OpenAI 模型所必需的
_set_env("OPENAI_API_KEY")
# 设置 OpenAI API代理地址 (例如：https://api.apiyi.com/v1）
_set_env("OPENAI_BASE_URL")

OPENAI_API_KEY: ··········
OPENAI_BASE_URL: ··········


[这里](https://python.langchain.com/v0.2/docs/how_to/#chat-models)是一个有用的指南，介绍了您可以使用聊天模型做的所有事情，但我们在下面会展示一些重点。如果您按照 README 中的说明运行了 `pip install -r requirements.txt`，那么您已经安装了 `langchain-openai` 包。有了这个包，我们可以实例化我们的 `ChatOpenAI` 模型对象。如果您是第一次注册 API，您应该会收到[免费积分](https://community.openai.com/t/understanding-api-limits-and-free-tier/498517)，可以应用于任何模型。您可以[在这里](https://openai.com/api/pricing/)查看各种模型的定价。笔记本将默认使用 `gpt-4o`，因为它在质量、价格和速度之间取得了良好的平衡[更多信息请参见这里](https://help.openai.com/en/articles/7102672-how-can-i-access-gpt-4-gpt-4-turbo-gpt-4o-and-gpt-4o-mini)，但您也可以选择价格较低的 `gpt-3.5` 系列模型。

聊天模型有几个[标准参数](https://python.langchain.com/v0.2/docs/concepts/#chat-models)可以设置。最常见的两个是：

* `model`：模型名称
* `temperature`：采样温度

`Temperature` 控制模型输出的随机性或创造性，其中低温度（接近 0）产生更确定性和专注的输出。这适合需要准确性或事实性响应的任务。高温度（接近 1）适合创造性任务或生成多样化响应。

In [7]:
# 导入 ChatOpenAI 类
from langchain_openai import ChatOpenAI

# 创建 GPT-4o 聊天模型实例
# model="gpt-4o": 使用 GPT-4o 模型，这是 OpenAI 的最新模型
# temperature=0: 设置温度为 0，使输出更加确定性和一致
gpt4o_chat = ChatOpenAI(model="gpt-4o", temperature=0)

# 创建 GPT-3.5 Turbo 聊天模型实例
# model="gpt-3.5-turbo-0125": 使用 GPT-3.5 Turbo 模型（2025年1月版本）
# temperature=0: 同样设置为 0，确保输出的一致性
gpt35_chat = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

LangChain 中的聊天模型有许多[默认方法](https://python.langchain.com/v0.2/docs/concepts/#runnable-interface)。在大多数情况下，我们将使用：

* `stream`：流式返回响应的块
* `invoke`：在输入上调用链

如前所述，聊天模型接受[消息](https://python.langchain.com/v0.2/docs/concepts/#messages)作为输入。消息具有一个角色（描述谁在说消息）和一个内容属性。我们稍后会更多地讨论这个问题，但这里让我们先展示基础知识。

In [14]:
# 导入 HumanMessage 类，用于创建人类用户的消息
from langchain_core.messages import HumanMessage

# 创建一个人类消息
# content: 消息内容
# name: 发送者的名称（可选）
msg = HumanMessage(content="你好呀", name="萤火AI百宝箱")

# 创建消息列表
# 聊天模型通常接受消息列表作为输入
messages = [msg]

# 使用消息列表调用模型
# invoke() 方法会发送消息给模型并返回响应
gpt4o_chat.invoke(messages)

AIMessage(content='你好！有什么我可以帮助你的吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 17, 'total_tokens': 27, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'input_tokens': 0, 'output_tokens': 0, 'input_tokens_details': None}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_5d58a6052a', 'id': 'chatcmpl-CDo2rrzfzFs9YulezWYT2KspS9lxW', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--c4ea9394-a482-48be-97e8-c817e5178569-0', usage_metadata={'input_tokens': 17, 'output_tokens': 10, 'total_tokens': 27, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

我们得到一个 `AIMessage` 响应。另外，请注意我们可以直接用字符串调用聊天模型。当字符串作为输入传递时，它会被转换为 `HumanMessage`，然后传递给底层模型。


In [15]:
# 直接使用字符串调用模型
# 当传入字符串时，LangChain 会自动将其转换为 HumanMessage
gpt4o_chat.invoke("LangChain学习路线")

AIMessage(content='学习LangChain的路线可以根据你的背景和目标进行调整。以下是一个通用的学习路线，适合对自然语言处理和人工智能有一定了解的学习者：\n\n### 1. 基础知识\n- **Python编程**：确保你对Python有基本的理解，因为LangChain是基于Python的。\n- **自然语言处理（NLP）基础**：了解基本的NLP概念，如分词、词性标注、命名实体识别等。\n- **机器学习基础**：熟悉基本的机器学习概念和框架，如TensorFlow或PyTorch。\n\n### 2. 理解LangChain\n- **LangChain简介**：了解LangChain的基本概念、目标和应用场景。\n- **安装和设置**：学习如何安装LangChain及其依赖项。\n\n### 3. LangChain核心概念\n- **链（Chain）**：学习如何构建和使用链来处理复杂的NLP任务。\n- **提示（Prompt）**：了解如何设计和优化提示以获得更好的模型输出。\n- **记忆（Memory）**：学习如何在对话中使用记忆来保持上下文。\n- **代理（Agent）**：了解如何使用代理来处理动态任务和决策。\n\n### 4. 实践与应用\n- **示例项目**：通过官方文档或社区资源中的示例项目，实践LangChain的使用。\n- **自定义应用**：尝试构建一个简单的应用，如聊天机器人或文本生成工具。\n- **集成与部署**：学习如何将LangChain应用集成到现有系统中，并进行部署。\n\n### 5. 深入学习与优化\n- **高级功能**：探索LangChain的高级功能，如自定义链、复杂的代理策略等。\n- **性能优化**：学习如何优化LangChain应用的性能，包括提示优化和模型选择。\n\n### 6. 社区与资源\n- **参与社区**：加入LangChain的社区，如GitHub、论坛或Slack，获取最新资讯和支持。\n- **持续学习**：关注LangChain的更新和新功能，保持学习和实践。\n\n通过以上步骤，你可以逐步掌握LangChain的使用，并能够应用于实际项目中。记得根据自己的进度和需求调整学习计划。', additional_kwargs={'refusal': 

In [16]:
# 使用 GPT-3.5 Turbo 模型调用
# 同样可以直接传入字符串
gpt35_chat.invoke("LangChain学习路线")

AIMessage(content='LangChain学习路线是一个逐步学习编程语言的过程，可以帮助你逐步掌握编程语言的基础知识和技能。以下是一个可能的LangChain学习路线：\n\n1. 学习基础知识：首先，你需要学习编程语言的基础知识，包括语法、数据类型、变量、运算符等。你可以通过阅读教科书、参加在线课程或观看教学视频来学习这些知识。\n\n2. 练习编程：一旦你掌握了基础知识，你可以开始练习编程。你可以通过编写简单的程序来巩固所学知识，并逐渐提高难度。\n\n3. 学习高级特性：一旦你熟悉了编程语言的基础知识，你可以开始学习一些高级特性，如函数、类、模块等。这些特性可以帮助你编写更复杂的程序。\n\n4. 实践项目：最后，你可以通过实践项目来应用所学知识。你可以选择一个感兴趣的主题，编写一个完整的程序，并将其部署到实际环境中。\n\n通过以上学习路线，你可以逐步掌握编程语言的基础知识和技能，并成为一名优秀的程序员。祝你学习顺利！', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 404, 'prompt_tokens': 14, 'total_tokens': 418, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'input_tokens': 0, 'output_tokens': 0, 'input_tokens_details': None}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_0165350fbb', 'id': 'chatcmpl-CDo3PsHiW4twU824Yk4VIiRiyzSaX', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--6a480cf1-d9b2-40d7-897f-5591fb47fb6a-0', usage_metadata={'input_tokens': 14, 'output_tokens': 404

所有聊天模型的接口都是一致的，模型通常在每个笔记本启动时初始化一次。

因此，如果您强烈偏好另一个提供商，您可以轻松地在模型之间切换，而无需更改下游代码。


## 搜索工具

您还会在 README 中看到 [Tavily](https://tavily.com/)，这是一个为 LLM 和 RAG 优化的搜索引擎，旨在提供高效、快速和持久的搜索结果。如前所述，注册很容易，并提供慷慨的免费层级。一些课程（在模块 4 中）将默认使用 Tavily，但当然，如果您想为自己修改代码，也可以使用其他搜索工具。

In [17]:
# 设置 Tavily API 密钥
# 这是使用 Tavily 搜索功能所必需的
_set_env("TAVILY_API_KEY")

In [19]:
# 导入 Tavily 搜索结果工具
from langchain_community.tools.tavily_search import TavilySearchResults

# 创建 Tavily 搜索实例
# max_results=3: 设置最大返回结果数量为 3
tavily_search = TavilySearchResults(max_results=3)

# 执行搜索
# 搜索 "What is LangGraph?" 并获取结果
search_docs = tavily_search.invoke("通俗解释LangGraph?包含应用场景，其他竞品对比分析，企业落地")

In [20]:
# 显示搜索结果
# 这将输出搜索到的文档列表，包含 URL 和内容摘要
search_docs

[{'title': '从零构建企业级多智能体系统：LangGraph 全流程实战+ 部署指南 ...',
  'url': 'https://blog.csdn.net/sinat_28461591/article/details/147076125',
  'content': 'LangGraph 是 LangChain 生态中的高阶 API 框架，专为多角色协作 Agent 设计，核心思想是将复杂工作流分解为有向图结构，节点为原子操作，边为执行逻辑。与传统工作流引擎相比，LangGraph 支持 Python 函数/LLM 调用、内存共享状态管理和 LLM 实时决策，并集成 LangSmith 可视化调试工具。其架构演进从 2023 年 Q4 的原型设计到 2024 年 Q3 的多租户线程管理，逐步完善。LangGraph 通过增量更新策略优化状态管理，支持持久化检查点和多租户线\n\n革新！AI应用架构师助力数据分析师AI智能体在智能化数据洞察中革新\n\n最新发布\n\nAI天才研究院\n\n08-02\n\n927 [...] 一、LangGraph 是什么？为什么说它重新定义了 Agent 编排\n 二、核心原理解析：图结构 + 状态驱动，才是智能体的最佳形式\n 三、实战项目：打造一个企业级多角色 Agent 系统（含工具链）\n  + 3.1 项目目标与场景模拟\n  + 3.2 项目架构图与核心流程设计\n  + 3.3 代码实战：节点构建与图结构编排\n  + 3.4 工具调用与图表生成节点实战\n  + 3.5 前后端部署与接口封装实战\n  + 3.6 部署建议与工程总结\n 四、部署实战：如何将 LangGraph 系统跑在本地或线上？\n 五、AutoGen vs LangGraph：哪个才适合生产环境？\n 六、最佳实践建议 + 常见问题汇总\n 七、总结：为什么 LangGraph 是 Agent 工程落地的必经之路\n\n## 一、LangGraph 是什么？为什么说它重新定义了 Agent 编排\n\n> 单个模型 ≠ 有用的产品，真正复杂的智能体系统，需要 任务分解、角色分工、状态管理和异常回滚。\n\nLangChain 虽然为我们提供了丰富的工具封装和链式调用接口，但当面对以下场景时，它开始显得力不从心： [...] 多